In [ ]:
!conda create -n newCondaEnvironment -c cctbx202208 python=3.9 -y
!source /opt/conda/bin/activate newCondaEnvironment && conda install -c cctbx202208 python -y

In [ ]:
!/opt/conda/envs/newCondaEnvironment/bin/python3 --version
!echo 'print("Hello, World!")' > test.py
!/opt/conda/envs/newCondaEnvironment/bin/python3 test.py

In [ ]:
pip install jupyter

In [ ]:
!sudo rm /opt/conda/bin/python3
!sudo ln -sf /opt/conda/envs/newCondaEnvironment/bin/python3 /opt/conda/bin/python3

In [ ]:
!sudo rm /opt/conda/bin/python3.10
!sudo ln -sf /opt/conda/envs/newCondaEnvironment/bin/python3 /opt/conda/bin/python3.10

In [ ]:
!sudo rm /opt/conda/bin/python
!sudo ln -s /opt/conda/envs/newCondaEnvironment/bin/python3 /opt/conda/bin/python

In [ ]:
!python --version

In [ ]:
!pip install -r /kaggle/input/requirements/requirements.pip

In [ ]:
!pip install opencv-contrib-python==4.4.0.46 numpy==1.19.3

# Parameters

In [ ]:
DATA_FOLDER = '/kaggle/input/data'
RESULTS_FOLDER = '/kaggle/working/results'
TENSORBOARD_BASE_FOLDER = '/kaggle/working/tensorboard'
INDICES_FILE = '/kaggle/input/data/Data_Entry_2017.csv'

WEIGHT_FILE_NAME = "weights.best.hdf5"

MIN_CASES = 1000

IMG_SIZE = (299, 299)
VGG19_IMG_SIZE = (224,224)
MOBILENET_IMG_SIZE = (224,224)
INCEPTIONRESNETV2_IMG_SIZE= (299,299)
MOBILENETV2_IMG_SIZE = (224,224)
LARGE_IMG_SIZE = (512,512)


LEARNING_RATE = 0.0005
SYNTHETIC_BATCH_SIZE = 256
BATCH_SIZE = 16
DEFAULT_OPTIMIZER = 'adam'
ACCUMULATION_STEPS = int(SYNTHETIC_BATCH_SIZE / BATCH_SIZE)
VALIDATION_BATCH_SIZE = BATCH_SIZE*2
EPOCHS = 5
EARLY_STOPPING_PATIENCE = 2
RL_PLATEAU_PATIENCE = 2
WORKERS = 8

# Reset

In [ ]:
import tensorflow as tf
import gc

# Reset Keras Session
def reset_keras():
  
    sess = tf.compat.v1.get_default_session()
    if sess is not None:
        tf.compat.v1.keras.backend.clear_session()
        sess.close()

    print(gc.collect())  # If it's done something, you should see a number being outputted

    # Configure the GPU memory options (if needed)
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))


# Data Preparation

In [ ]:
from itertools import chain
import numpy as np
import pandas as pd
import os
from glob import glob
from sklearn.model_selection import train_test_split

all_labels = None

def load_metadata(data_folder=DATA_FOLDER,
                  metadata_file=INDICES_FILE):
   
    metadata = pd.read_csv(os.path.join(data_folder, metadata_file))
    file_system_scan = {os.path.basename(x): x for x in
                        glob(os.path.join(data_folder, 'images*', 'images','*.png'))}
    if len(file_system_scan) != metadata.shape[0]:
        raise Exception(
            'ERROR: Different number metadata records and png files.'.format())

    metadata['path'] = metadata['Image Index'].map(file_system_scan.get)
    print('Total x-ray records:{}.'.format((metadata.shape[0])))

    return metadata


def preprocess_metadata(metadata, minimum_cases=MIN_CASES):


    metadata['Finding Labels'] = metadata['Finding Labels'].map(
        lambda x: x.replace('No Finding', ''))

    labels = np.unique(
        list(chain(*metadata['Finding Labels'].map(lambda x: x.split('|')).tolist())))
    labels = [x for x in labels if len(x) > 0]

    for c_label in labels:
        if len(c_label) > 1:  # leave out empty labels
            metadata[c_label] = metadata['Finding Labels'].map(
                lambda finding: 1.0 if c_label in finding else 0)

    labels = [c_label for c_label in labels if metadata[c_label].sum()
              > minimum_cases]

    sample_weights = metadata['Finding Labels'].map(
        lambda x: len(x.split('|')) if len(x) > 0 else 0).values + 4e-2
    sample_weights /= sample_weights.sum()
    metadata = metadata.sample(80000, weights=sample_weights)

    labels_count = [(c_label, int(metadata[c_label].sum()))
                    for c_label in labels]

    print('Labels ({}:{})'.format((len(labels)), (labels_count)))
    print('Total x-ray records:{}.'.format((metadata.shape[0])))

    all_labels = labels
    
    return metadata, labels


def stratify_train_test_split(metadata):
   
    stratify = metadata['Finding Labels'].map(lambda x: x[:4])
    train, valid = train_test_split(metadata,
                                    test_size=0.25,
                                    random_state=2018,
                                    stratify=stratify)
    return train, valid


# Gradient Accumulation

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Optimizer

class AdamAccumulate(Optimizer):
    def __init__(self, lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0., amsgrad=False, accum_iters=1, **kwargs):
        if accum_iters < 1:
            raise ValueError('accum_iters must be >= 1')
        super(AdamAccumulate, self).__init__(**kwargs)
        self.lr = lr
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.amsgrad = amsgrad
        self.accum_iters = accum_iters
        self.accum_iters_float = float(accum_iters)

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = []

        completed_updates = tf.floor_div(tf.cast(self.iterations, tf.float32), self.accum_iters_float)

        lr = self.lr

        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * completed_updates))

        t = completed_updates + 1

        lr_t = lr * (tf.sqrt(1. - tf.pow(self.beta_2, t)) / (1. - tf.pow(self.beta_1, t)))

        update_switch = tf.equal((self.iterations + 1) % self.accum_iters, 0)
        update_switch = tf.cast(update_switch, tf.float32)

        ms = [tf.zeros(tf.shape(p), dtype=p.dtype) for p in params]
        vs = [tf.zeros(tf.shape(p), dtype=p.dtype) for p in params]
        gs = [tf.zeros(tf.shape(p), dtype=p.dtype) for p in params]

        if self.amsgrad:
            vhats = [tf.zeros(tf.shape(p), dtype=p.dtype) for p in params]
        else:
            vhats = [tf.zeros(1, dtype=p.dtype) for p in params]

        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat, tg in zip(params, grads, ms, vs, vhats, gs):
            sum_grad = tg + g
            avg_grad = sum_grad / self.accum_iters_float

            m_t = (self.beta_1 * m) + (1. - self.beta_1) * avg_grad
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * tf.square(avg_grad)

            if self.amsgrad:
                vhat_t = tf.maximum(vhat, v_t)
                p_t = p - lr_t * m_t / (tf.sqrt(vhat_t) + self.epsilon)
                self.updates.append(tf.assign(vhat, (1 - update_switch) * vhat + update_switch * vhat_t))
            else:
                p_t = p - lr_t * m_t / (tf.sqrt(v_t) + self.epsilon)

            self.updates.append(tf.assign(m, (1 - update_switch) * m + update_switch * m_t))
            self.updates.append(tf.assign(v, (1 - update_switch) * v + update_switch * v_t))
            self.updates.append(tf.assign(tg, (1 - update_switch) * sum_grad))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(tf.assign(p, (1 - update_switch) * p + update_switch * new_p))
        return self.updates

    def get_config(self):
        config = {'lr': self.lr, 'beta_1': self.beta_1, 'beta_2': self.beta_2, 'decay': self.decay, 'epsilon': self.epsilon, 'amsgrad': self.amsgrad}
        base_config = super(AdamAccumulate, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))




# Utils

In [ ]:

import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

from sklearn.metrics import classification_report, confusion_matrix

def calculate_accuracies(results, labels=None, run_timestamp='unspecified'):
   
    if labels is None:
        return

    print(classification_report(labels, results))
    clf_report = classification_report(labels, results, output_dict=True)
    clf_filename = f'classification_report_{run_timestamp}.json'
    clf_path = os.path.join(RESULTS_FOLDER, clf_filename)
    df = pd.DataFrame(clf_report)
    df.to_json(clf_path, orient='columns')
    print(f'Classification report saved to {clf_path}')

    conf_matrix = confusion_matrix(labels, results)
    conf_matrix_filename = f'confusion_matrix_{run_timestamp}.csv'
    conf_matrix_path = os.path.join(RESULTS_FOLDER,
                                    conf_matrix_filename)
    df = pd.DataFrame(conf_matrix)
    df.to_csv(conf_matrix_path, index=True)
    print(f'Confusion matrix saved to {conf_matrix_path}')

    
def plot_train_metrics(model_history, model_name, results_folder, run_timestamp='unspecified'):

    train_losses = model_history.history['loss']
    val_losses = model_history.history['val_loss']
    final_val_loss = val_losses[-1]

    train_acc = model_history.history['binary_accuracy']
    val_acc = model_history.history['val_binary_accuracy']
    final_val_acc = val_acc[-1]

    loss_filename = f'{model_name}_loss_plot_val_loss_{final_val_loss:.4f}_val_acc_{final_val_acc:.4f}_{run_timestamp}.png'
    loss_fig_path = os.path.join(results_folder, loss_filename)
    acc_filename = f'{model_name}_accuracy_plot_val_loss_{final_val_loss:.4f}_val_acc_{final_val_acc:.4f}_{run_timestamp}.png'
    acc_fig_path = os.path.join(results_folder, acc_filename)

    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.xticks(np.arange(0, len(train_losses), step=1))
    plt.xlabel('Epoch Number')
    plt.ylabel('Loss')
    plt.title(f'{model_name} Model\nRun time: {run_timestamp}')
    plt.legend(('Training', 'Validation'))
    plt.savefig(loss_fig_path)

    # clear axes and figure to reset for next plot
    plt.cla()
    plt.clf()

    # generate and save accuracy plot
    plt.plot(train_acc)
    plt.plot(val_acc)
    plt.xticks(np.arange(0, len(train_acc), step=1))
    plt.xlabel('Epoch Number')
    plt.ylabel('Accuracy')
    plt.title(f'{model_name} Model\nRun time: {run_timestamp}')
    plt.legend(('Training', 'Validation'))
    plt.savefig(acc_fig_path)

    return loss_fig_path, acc_fig_path


def save_model(model, model_history, model_name, results_folder, run_timestamp='unspecified'):
   
    final_val_loss = model_history.history['val_loss'][-1]
    final_val_acc = model_history.history['val_binary_accuracy'][-1]

    json_filename = f'{model_name}_config_val_loss_{final_val_loss:.4f}_val_acc_{final_val_acc:.4f}_{run_timestamp}.json'
    output_json = os.path.join(
        results_folder, json_filename)
    with open(output_json, 'w') as json_file:
        json_file.write(model.to_json())

    weights_filename = f'{model_name}_weights_val_loss_{final_val_loss:.4f}_val_acc_{final_val_acc:.4f}_{run_timestamp}.hdf5'
    output_weights = os.path.join(results_folder, weights_filename)
    model.save_weights(output_weights)

    symlink_path = os.path.join(
        results_folder, f'{model_name}_weights_latest.hdf5')
    try:
        os.symlink(output_weights, symlink_path)
    except FileExistsError:
        os.remove(symlink_path)
        os.symlink(output_weights, symlink_path)
    print(f'Created symbolic link to final weights -> {symlink_path}')

    return output_json, output_weights

# Train

In [ ]:

import os
from time import time
import datetime

from time import time
import datetime

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.mobilenet import preprocess_input as MobileNet_preprocess_input
from tensorflow.keras.applications.vgg19 import preprocess_input as VGG19_preprocess_input
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as InceptionResNetV2_preprocess_input
from tensorflow.keras.layers import Flatten, Dense, LocallyConnected2D, MaxPooling2D, Input, Conv2D, AvgPool2D, Lambda, Dropout, Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, multiply, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np
from sklearn.metrics import roc_curve, auc

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
predictionList=[]
AUC_SCORES=[]
TEST_Y=[]
LABELS=[]
RUN_TIMESTAMP = datetime.datetime.now().isoformat('-')
attention_models = []


base_models = [
    [VGG19, VGG19_IMG_SIZE, VGG19_preprocess_input],
    [MobileNet, MOBILENET_IMG_SIZE, MobileNet_preprocess_input],[InceptionResNetV2, INCEPTIONRESNETV2_IMG_SIZE,
     InceptionResNetV2_preprocess_input],
]


def create_data_generator(dataset,
                          labels,
                          batch_size,
                          preprocessing_function,
                          color_mode="rgb",
                          target_size=IMG_SIZE):
   
    dataset['newLabel'] = dataset.apply(
        lambda x: x['Finding Labels'].split('|'), axis=1)

    image_generator = ImageDataGenerator(samplewise_center=True,
                                         samplewise_std_normalization=True,
                                         horizontal_flip=True,
                                         vertical_flip=False,
                                         height_shift_range=0.05,
                                         width_shift_range=0.1,
                                         rotation_range=5,
                                         shear_range=0.1,
                                         fill_mode='reflect',
                                         zoom_range=0.15,
                                         preprocessing_function=preprocessing_function)

    dataset_generator = image_generator.flow_from_dataframe(dataframe=dataset,
                                                            directory=None,
                                                            x_col='path',
                                                            y_col='newLabel',
                                                            class_mode='categorical',
                                                            classes=labels,
                                                            target_size=target_size,
                                                            color_mode=color_mode,
                                                            batch_size=batch_size)

    return dataset_generator


def _create_attention_model(frozen_model, labels, optimizer='adam'):
   
    frozen_features = Input(frozen_model.get_output_shape_at(0)[
        1:], name='feature_input')
    frozen_depth = frozen_model.get_output_shape_at(0)[-1]
    new_features = BatchNormalization()(frozen_features)

    attention_layer = Conv2D(128, kernel_size=(1, 1), padding='same',
                             activation='elu')(new_features)
    attention_layer = Conv2D(32, kernel_size=(1, 1), padding='same',
                             activation='elu')(attention_layer)
    attention_layer = Conv2D(16, kernel_size=(1, 1), padding='same',
                             activation='elu')(attention_layer)
    attention_layer = AvgPool2D((2, 2), strides=(1, 1), padding='same')(
        attention_layer)  # smooth results
    attention_layer = Conv2D(1,
                             kernel_size=(1, 1),
                             padding='valid',
                             activation='sigmoid')(attention_layer)

    up_c2_w = np.ones((1, 1, 1, frozen_depth))
    up_c2 = Conv2D(frozen_depth, kernel_size=(1, 1), padding='same',
                   activation='linear', use_bias=False, weights=[up_c2_w])
    up_c2.trainable = False
    attention_layer = up_c2(attention_layer)

    mask_features = multiply([attention_layer, new_features])
    gap_features = GlobalAveragePooling2D()(mask_features)
    gap_mask = GlobalAveragePooling2D()(attention_layer)

    gap = Lambda(lambda x: x[0]/x[1],
                 name='RescaleGAP')([gap_features, gap_mask])
    gap_dr = Dropout(0.5)(gap)
    dr_steps = Dropout(0.5)(Dense(128, activation='elu')(gap_dr))
    out_layer = Dense(len(labels), activation='sigmoid')(dr_steps)

    attention_model = Model(inputs=[frozen_features], outputs=[
        out_layer], name='attention_model')

    attention_model.compile(optimizer=optimizer, loss='binary_crossentropy',
                            metrics=['binary_accuracy'])

    return attention_model


def _create_base_model(Model, labels, input_shape, trainable=False, weights="imagenet"):
    
    base_model = Model(weights=weights,
                       include_top=False,
                       input_shape=input_shape)
    base_model.trainable = trainable
    base_model(tf.keras.Input(input_shape))
    return base_model


def create_simple_model(base_model, labels, optimizer='adam'):
  

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(512))
    model.add(Dropout(0.5))
    model.add(Dense(len(labels), activation='sigmoid'))
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['binary_accuracy', 'mae'])
    print(f'{model.summary()}')
    return model


def create_attention_model(base_model, labels, optimizer='adam'):
   

    attention_model = _create_attention_model(
        base_model, labels, optimizer=optimizer)

    model = Sequential(name='combined_model')
    model.add(base_model)
    model.add(attention_model)
    model.compile(optimizer=optimizer, loss='binary_crossentropy',
                  metrics=['binary_accuracy'])
    
    attention_models.append(model)

    print(f'{model.summary()}')

    return model


def fit_model(model, model_name, train, valid):
   
    results_folder = os.path.join(RESULTS_FOLDER, model_name)
    if not os.path.exists(results_folder):
        os.makedirs(results_folder)

    weight_path = os.path.join(results_folder, WEIGHT_FILE_NAME)

    checkpoint = ModelCheckpoint(weight_path,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='min',
                                 save_weights_only=True)

    early = EarlyStopping(monitor="val_loss",
                          mode="min",
                          patience=EARLY_STOPPING_PATIENCE)

    tensorboard = TensorBoard(log_dir=os.path.join(
        RESULTS_FOLDER, TENSORBOARD_BASE_FOLDER, model_name))

    dynamicLR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                  patience=2, min_lr=LEARNING_RATE/100)

    callbacks_list = [tensorboard, checkpoint, dynamicLR, early]

    history = model.fit(  train,
                          validation_data=valid,
                          validation_steps=valid.samples//valid.batch_size,
                          steps_per_epoch=train.samples//train.batch_size,
                          epochs=EPOCHS,
                          callbacks=callbacks_list,
                          use_multiprocessing=True,
                          workers=WORKERS)

    loss_fig_path, acc_fig_path = plot_train_metrics(
        history, model_name, results_folder,  RUN_TIMESTAMP)
    print(f'Saved loss plot -> {loss_fig_path}')
    print(f'Saved accuracy plot -> {acc_fig_path}')

    json_path, weights_path = save_model(
        model, history, model_name, results_folder, RUN_TIMESTAMP)
    print(f'Saved json config -> {json_path}')
    print(f'Saved weights -> {weights_path}')

    return model


def plot_ROC(labels, test_Y, pred_Y, model_name):
    aucScores = []
    fig, c_ax = plt.subplots(1, 1, figsize=(9, 9))
    for (idx, c_label) in enumerate(labels):
        fpr, tpr, thresholds = roc_curve(
            test_Y[:, idx].astype(int), pred_Y[:, idx])
        c_ax.plot(fpr, tpr, label='%s (AUC:%0.2f)' % (c_label, auc(fpr, tpr)))
        aucScores.append(auc(fpr,tpr))
    c_ax.legend()
    c_ax.set_title(model_name+' ROC Curve')
    c_ax.set_xlabel('False Positive Rate')
    c_ax.set_ylabel('True Positive Rate')

    ROC_image_file_path = os.path.join(
        RESULTS_FOLDER, model_name, model_name + '_ROC.png')
    AUC_SCORES.append(aucScores)
    fig.savefig(ROC_image_file_path)
    print('Saved ROC plot at'+ROC_image_file_path)


def train_model(_Model, input_shape, transfer_learing,
                preprocessing_function,
                train, valid, labels,
                extend_model_callback, optimizer,
                name_prefix, weights="imagenet"):

    if not transfer_learing:
        weights = None

    train_generator = create_data_generator(
        train, labels, BATCH_SIZE, preprocessing_function, target_size=input_shape)
    validation_generator = create_data_generator(
        valid, labels, VALIDATION_BATCH_SIZE, preprocessing_function, target_size=input_shape)

    test_X, test_Y = next(validation_generator)

    baseModel = _create_base_model(_Model,
                                   labels,
                                   test_X.shape[1:],
                                   trainable=not transfer_learing,
                                   weights=weights)

    model = extend_model_callback(baseModel, labels, optimizer)
    model_name = f'{name_prefix}_{baseModel.name}'

    model = fit_model(model, model_name,
                      train_generator, validation_generator)

    # print ROC
    test_X, test_Y = next(create_data_generator(
        valid, labels, 10000, None, target_size=input_shape))
    pred_Y = model.predict(test_X, batch_size=32, verbose=True)
    TEST_Y.append(test_Y)
    predictionList.append(pred_Y)
    
    plot_ROC(labels, test_Y, pred_Y, model_name)


def plot_model_ROC(_Model, input_shape, transfer_learing,
                preprocessing_function,
                train, valid, labels,
                extend_model_callback, optimizer,
                name_prefix, weights="imagenet"):

    test_X, test_Y = next(create_data_generator(
        valid, labels, 10000, None, target_size=input_shape))

    baseModel = _create_base_model(_Model,
                                   labels,
                                   test_X.shape[1:],
                                   trainable=False,
                                   weights=None)

    model = extend_model_callback(baseModel, labels, optimizer)

    model_name = name_prefix+'_' + baseModel.name

    weights = os.path.join(RESULTS_FOLDER,
                           model_name, 'weights.best.hdf5')

    print('Loading '+weights)
    model.load_weights(weights, by_name=True)
    model.trainable = False

    pred_Y = model.predict(test_X, batch_size=32, verbose=True)

    plot_ROC(labels, test_Y, pred_Y, model_name)

def loop_in_combinations(callback, image_size=None, transfer_learing=True, use_preprocess_input=False):


    metadata = load_metadata()
    metadata, labels = preprocess_metadata(metadata)
    train, valid = stratify_train_test_split(metadata)
    LABELS.append(labels)
    # for these image sizes, we don't need gradient_accumulation to achieve BATCH_SIZE = 256
    optimizer = 'adam'
    if DEFAULT_OPTIMIZER != optimizer:
        optimizer = AdamAccumulate(
            lr=LEARNING_RATE, accum_iters=ACCUMULATION_STEPS)

    unfrozen = 'unfrozen_'
    if transfer_learing:
        unfrozen = ''
    custom_layers = [
        [create_attention_model, unfrozen+'Attention'],
    ]

    for [custome_layer, name_prefix] in custom_layers:
        for [_Model, input_shape, preprocess_input] in base_models:
            _image_size = image_size
            if _image_size is None:
                _image_size = input_shape
            _preprocess_input = preprocess_input
            if not use_preprocess_input:
                _preprocess_input = None
            callback(_Model, _image_size, transfer_learing, _preprocess_input,
                        train, valid, labels,
                        custome_layer, optimizer, name_prefix)

def plot_ROCs(image_size=None, transfer_learing=True, use_preprocess_input=False):

    loop_in_combinations(plot_model_ROC, image_size, transfer_learing, use_preprocess_input)

def train_multiple_networks(image_size=None, transfer_learing=True, use_preprocess_input=False):
   
    loop_in_combinations(train_model, image_size, transfer_learing, use_preprocess_input)


reset_keras()
train_multiple_networks(use_preprocess_input = True)

# Ensemble Method

In [ ]:
test_Y=TEST_Y[1]
all_labels=LABELS[0]
weightList = [0.1, 0.8, 0.1]

In [ ]:
def createROC(all_labels, test_Y, pred_Y, filename="roc.png"):
    aucScores = []
    fig, c_ax = plt.subplots(1,1, figsize = (9, 9))
    for (idx, c_label) in enumerate(all_labels):
        fpr, tpr, thresholds = roc_curve(test_Y[:,idx].astype(int), pred_Y[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
        aucScores.append(auc(fpr,tpr))
    c_ax.legend()
    c_ax.set_xlabel('False Positive Rate')
    c_ax.set_ylabel('True Positive Rate')
    fig.savefig(filename)
    return aucScores

In [ ]:
def MaxVoteEnsemble(predictionList):
    ensemble_preds = np.maximum(*predictionList)
    return ensemble_preds

def SimpleAverageEnsemble(predictionList):
    ensemble_preds = np.zeros(predictionList[0].shape)
    for pred in predictionList:
        ensemble_preds += pred
    
    ensemble_preds /= len(predictionList)
    return ensemble_preds

def WeightedAverageEnsemble(predictionList, weightList):
    ensemble_preds = np.zeros(predictionList[0].shape)
    for pred, weight in zip(predictionList, weightList):
        ensemble_preds += pred*weight
    return ensemble_preds

In [ ]:
max_vote_ensemble_preds = MaxVoteEnsemble(predictionList)
simple_ensemble_preds = SimpleAverageEnsemble(predictionList)
weighted_ensemble_preds = WeightedAverageEnsemble(predictionList, weightList)

In [ ]:
maxvoteensembleAUC = createROC(all_labels, test_Y, max_vote_ensemble_preds, filename="max_vote_ensemble_roc.png")
weightedensembleAUC = createROC(all_labels, test_Y, weighted_ensemble_preds, filename="weighted_ensemble_roc.png")
simpleensembleAUC = createROC(all_labels, test_Y, simple_ensemble_preds, filename="simple_ensemble_roc.png")

In [ ]:
summaryDF = pd.DataFrame(
    {'Class':all_labels,
    'Mobilenet': AUC_SCORES[1],
    'Resnet': AUC_SCORES[2],
    'VGG': AUC_SCORES[0],
    'Avg Ensemble': simpleensembleAUC,
    'Wght Avg Ensemble': weightedensembleAUC,
    'Max Vote Ensemble': maxvoteensembleAUC}
)

In [ ]:
summaryDF.round(3)

In [ ]:
summaryDF.to_csv('my_data.csv', index=False)

In [ ]:
import csv
with open('predictionData.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(predictionList)

In [ ]:
with open('AUCData.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(AUC_SCORES)